In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import pandas as pd
from pathlib import Path
import os
import sys

In [4]:
in_path = "../source_data/Openstaande posten crediteuren 300423 SAP.XLSX"
out_path = "../validation_data/Openstaande_posten_crediteuren_300423_SAP.xlsx"

In [5]:
input_path = Path(in_path)
assert input_path.exists()
# NB: the read_excel function indirectly requires openpyxl as an Excel engine
in_df = pd.read_excel(input_path) 

In [6]:
assert input_path.exists()
in_df = pd.read_excel(input_path) 

In [8]:
grouper_column = in_df.columns[0]
grouper_values = in_df[grouper_column]
# Drop the "grouper_column" we don't need it anymore.
in_df = in_df.drop(columns=[grouper_column])

In [12]:
# Vectorize the columns to booleans
group_defined_list = list(grouper_values.isna())

In [13]:
index_rows_for_group = []
keys = []
results = {}

# For each entry in our boolean array
for index, is_group_defined in enumerate(group_defined_list):
    if is_group_defined is True:
        # Have we identified any keys?
        if len(keys) > 0:
            results[tuple(keys)] = index_rows_for_group
            keys = []
            index_rows_for_group = []
        index_rows_for_group.append(index)
    else:
        value = grouper_values[index]
        keys.append(value)

In [22]:
# Iterate through the results
# Building the df again with the subsets
subset_dfs = []
for value_key, df_indices in results.items():
    subset_df = in_df.loc[df_indices].copy()
    subset_df["group"] = [value_key] * len(subset_df)
    subset_dfs.append(subset_df)
sub_df = pd.concat(subset_dfs)

In [23]:
sub_df.columns

Index(['Zuordnung', 'Belegnummer', 'Belegart', 'Belegdatum',
       'Sonderhauptb.Kennz.', 'Symbol Nettofälligkeit',
       'Betrag in Hauswährung', 'Hauswährung', 'Ausgleichsbeleg', 'Text',
       'group'],
      dtype='object')

In [26]:
columns_to_drop = [
    "Belegart",
    "Sonderhauptb.Kennz.",
    "Symbol Nettofälligkeit",
    "Text",
    "Hauswährung",
    "Ausgleichsbeleg",
    "Zuordnung",
]

In [27]:
# Drop irrelevant columns
sub_df = sub_df.drop(columns=columns_to_drop)

In [29]:
sub_df.to_excel(out_path)